In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.io import wavfile
from scipy.fft import fftshift
from scipy.fftpack import fftfreq,fft
from scipy.fftpack import dct
from scipy import interpolate
from scipy import fft

import torch

from sklearn.preprocessing import Normalizer
scaler=Normalizer(norm='max')

In [3]:
def Coefs(fs,data,NFFT=512,num_ceps = 14,normalize = False,std_len = 96,only_mel=False,scaling=False):
    frame_size = 0.02 # 20мс
    frame_stride = 0.01 # шаг 10 мс
    frame_length  = int(round(fs*frame_size))
    frame_step = int(round(fs*frame_stride))

    signal_length = len(data)
    num_full_frames = int(np.ceil(float(np.abs(signal_length - frame_length )) / frame_step))  # Число фреймов, полностью заполненных полезным сигналом

    pad_signal_length = num_full_frames * frame_step + frame_length # длина исходного сигнала + нули в конце, чтобы стало целое число фремов
    z = np.zeros((pad_signal_length - signal_length)) #добавляем нули к последнему фрейму чтобы его заполнить
    pad_signal = np.append(data, z) # теперь в сигнале полное число фреймов, но последний дополняется нулями
    num_frames = num_full_frames + 1 # полное число фреймов

    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_full_frames * frame_step+1, frame_step), (frame_length, 1)).T #матрица индексов: строки -фреймы, столбы - элементы фрейма 
    frames = pad_signal[indices]

    # Хэмминг + Фурье
    T = 1.0 / fs
    w = np.hamming(frame_length) # Окно Хэмминга
    frames_fourier_w = np.abs(fft.rfft(frames*w, NFFT))       
    # Вычисляем периодограмму каждого фрейма ( спектральную мощность )
    P = ((frames_fourier_w) ** 2)/NFFT  # Power Spectrum


    # Вычисляем блок мел-фильтров(треугольных)
    nfilt=40
    low_freq_mel = 0
    high_freq_mel = (2595 * np.log10(1 + (fs / 2) / 700))  # Переводим герцы в мелы
    mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # равномерно распределены по мел шкале
    hz_points = (700 * (10**(mel_points / 2595) - 1))  # мелы в герцы
    bin = np.floor((NFFT + 1) * hz_points / fs)

    fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin[m - 1])   # left
        f_m = int(bin[m])             # center
        f_m_plus = int(bin[m + 1])    # right

        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])


    filter_banks = np.dot(P, fbank.T).T
    filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)  # Numerical Stability
    filter_banks = 20 * np.log10(filter_banks)  # dB
    
    if normalize:
        filter_banks = filter_banks - (np.mean(filter_banks, axis=1).reshape(-1,1) + 1e-8)
    
    if only_mel:
        # берем 14 коэфов
        coefs = filter_banks[1 : (num_ceps + 1),:]
    else:    
        #дискретное косинусное преобразование
        # число мел-кепстральных коэффов, остальные отбрасываем потому что они отражают быстрые изменения сигнала
        coefs = dct(filter_banks, type=2, axis=0, norm='ortho')[1 : (num_ceps + 1),:] # Берем 2-14

    #С помощью интерполяции формируем изображение необходимого размера
    x = np.arange(0,coefs.shape[1],1)
    y = np.arange(0,coefs.shape[0],1)

    f = interpolate.interp2d(x, y, coefs, kind='cubic')

    xnew = np.arange(0,coefs.shape[1],coefs.shape[1]/std_len)
    coefs = f(xnew, y)
    if scaling:
        coefs = scaler.fit_transform(coefs)
    return coefs

In [4]:
labels = pd.read_csv('D:/rech/VeriRu/words/labels.csv')
main_path = 'D:/rech/VeriRu/words/'
spisok = list(main_path + labels.idx.astype(str).values + '.wav')

In [5]:
labels

,idx,label
0,0,6
1,1,3
2,2,2
3,3,4
4,4,8
...,...,...
11637,11657,0
11638,11658,9
11639,11659,4
11640,11660,1


# Полносвязная 

In [8]:
for i in range(len(spisok)):
   
    fs, data = wavfile.read(spisok[i])
    mfcc_interp = Coefs(fs,data)
    if i==0:
        coefs = mfcc_interp.reshape(mfcc_interp.shape[1]*mfcc_interp.shape[0]) 
    else:
        coefs = np.vstack((coefs,mfcc_interp.reshape(mfcc_interp.shape[1]*mfcc_interp.shape[0])))
        
voice_numbers = np.hstack((coefs,labels.label.values.reshape(-1,1)))

np.random.shuffle(sounds)
np.savetxt("D:/rech/VeriRu/words/dataset_fc_no_ceps_96.csv", voice_numbers, delimiter=",")

# Сверточная

In [9]:
coefs = []
import torch
for i in range(len(spisok)):
    
    fs, data = wavfile.read(spisok[i])
    mfcc_interp = Coefs(fs,data)

    if i==0:
        coefs = torch.FloatTensor(mfcc_interp).unsqueeze(0)
    else:
        coefs = torch.cat((coefs,torch.FloatTensor(mfcc_interp).unsqueeze(0)))

conv_labels = torch.FloatTensor(labels.label.values.reshape(-1,1))
        
torch.save(coefs, 'D:/rech/VeriRu/words/dataset_conv_full_96.pt')
torch.save(conv_labels,'D:/rech/VeriRu/words/dataset_labels_full_96.pt')